# Quantum Transfer Learning (QTL) Demo
This notebook shows how to pretrain a small parameterized quantum circuit (PQC) on a source task, reuse it as a frozen feature extractor on a target task, and compare frozen vs fine-tuned training on the target task.

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from app import dataset
from app.transfer_pipeline import run_all_experiments
from app.feature_extractor import extract_features
from app.plots import plot_feature_space_source_vs_target, plot_training_curves_frozen_vs_finetune, plot_accuracy_comparison

# Create output directory for SVG plots
output_dir = Path('../examples')
output_dir.mkdir(exist_ok=True)

## 1. Generate source and target datasets
The source task uses two Gaussian blobs. The target task uses a shifted moons pattern to create a small domain shift.

In [ ]:
X_source, y_source = dataset.generate_source_dataset(n_samples=200)
X_target, y_target = dataset.generate_target_dataset(n_samples=200)
X_source.shape, X_target.shape

## 2. Run transfer learning experiments
We pretrain the PQC on the source task and then run two target-task strategies:
- **Frozen PQC + trainable head**
- **Fine-tune PQC + head**

In [ ]:
results = run_all_experiments(X_source, y_source, X_target, y_target, n_base_epochs=10, n_target_epochs=15)
results['frozen'], results['finetune']

## 3. Visualize feature spaces and training curves
All plots are saved as SVG to render cleanly on GitHub.

In [ ]:
base_params = results['finetune_params']
source_features = extract_features(X_source, base_params)
target_features = extract_features(X_target, base_params)
plot_feature_space_source_vs_target(source_features, y_source, target_features, y_target, output_path=str(output_dir / 'qtl_feature_space_source_vs_target.svg'))
plot_training_curves_frozen_vs_finetune(results['frozen_history']['head'], results['finetune_history']['joint'], output_path=str(output_dir / 'qtl_training_curves_frozen_vs_finetune.svg'))
plot_accuracy_comparison(results['frozen']['accuracy'], results['finetune']['accuracy'], output_path=str(output_dir / 'qtl_accuracy_comparison.svg'))
output_dir.iterdir()

## 4. Quick takeaways
- The frozen PQC reuses source-task knowledge and trains a light head quickly.
- Fine-tuning can further improve accuracy on the target task.
- SVG plots stay lightweight and diffable in GitHub.